### In this section, I will build an Agentic RAG

Before, I already used a medical-domain LLM to generate hypothentical questions for each chunk. 

The project's main purpose is medical Q&A.  So I am going to implement Multi-Vector as the foundation of the RAG system.

That means I am going to:
* Embedding those questions to vectorestore and put chunked documents to docstore.
* I will use UUID to be a link between vectorstore and docstore.

About the Agentic components:
* I will involve an agent to do reranking for the retrieved documents so that it can provide the most relevant context. 
* The agent can remember history conversation so that it can still retrieve the relevant documents in multi-turns conversation.
* The agent can genrate similar questions which will be used to retrieve documents.
* The agent can do function calling/MCP when there is no relevant document.

In [20]:
# Multi-Vector implementation
import os
import json
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from huggingface_hub import login

In [12]:
# This function will load the json file to json object
def load_json_list(path: str):    
    with open(path, mode = "r", encoding="utf-8") as f:
        return json.load(f)

In [13]:
workspace_base_path = os.getcwd()
dataset_path = os.path.join(workspace_base_path, "datasets", "medicine_data_hypothenticalquestions.json") 
print(dataset_path)

c:\Users\Montr\AI_Projects\MediPal\Agentic-RAG\datasets\medicine_data_hypothenticalquestions.json


In [14]:
data = load_json_list(dataset_path)

In [16]:
data[50:53]

[{'doc_id': '1b8794c4-5946-4a50-bf09-d920746db8cb',
  'questions': ['what special precautions should i follow about Chlordiazepoxide',
   'What should people who are pregnant or breastfeeding know about Chlordiazepoxide?',
   'What does the document say about drive car operate machinery until know how?'],
  'original_doc': 'before taking chlordiazepoxide,tell your doctor and pharmacist if you are allergic to chlordiazepoxide, alprazolam (xanax), clonazepam (klonopin), clorazepate (gen-xene, tranxene), diazepam (diastat, valium), estazolam, flurazepam, lorazepam (ativan), oxazepam, temazepam (restoril), triazolam (halcion), any other medications, or any of the ingredients in tablets and capsules. ask your pharmacist for a list of the ingredients.tell your doctor and pharmacist what prescription and nonprescription medications, vitamins, nutritional supplements, and herbal products you are taking or plan to take while taking chlordiazepoxide. your doctor may need to change the doses of y

In [3]:
with open("keys.txt") as f:
    os.environ["HF_TOKEN"] = f.read().strip()

# login using env var
login(os.environ["HF_TOKEN"])

print(f"Login Huggingface so that we can access the model")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Login Huggingface so that we can access the model


##### I choose sentence-transformers/embeddinggemma-300m-medical, as it is a sentence-transformers model finetuned from google/embeddinggemma-300m on the miriad/miriad-4.4M dataset (specifically the first 100.000 question-passage pairs from tomaarsen/miriad-4.4M-split). It maps sentences & documents to a 768-dimensional dense vector space and can be used for medical information retrieval, specifically designed for searching for passages (up to 1k tokens) of scientific medical papers using detailed medical questions.

* Reference: https://huggingface.co/sentence-transformers/embeddinggemma-300m-medical

In [4]:
embedding_model_id = "sentence-transformers/embeddinggemma-300m-medical"

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_id,
    model_kwargs = {'device': 'cpu'},
    # Normalizing helps cosine similarity behave better across models
    encode_kwargs={"normalize_embeddings": True},
)

You are trying to use a model that was created with Sentence Transformers version 5.2.0.dev0, but you're currently using version 5.1.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.


In [21]:
# The storage layer for the parent documents
docstore = InMemoryStore()
id_key = "doc_id"

In [22]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="medicinedocs", embedding_function=embeddings
)

In [23]:
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key=id_key,
)

In [18]:
Document("dd")

Document(metadata={}, page_content='dd')

In [30]:
doc_ids = []
questions = []
docs = []

In [31]:
for d in data[:50]:
    doc_id = d["doc_id"]
    doc_ids.append(doc_id)
    docs.append(Document(metadata={"doc_id": doc_id}, page_content=d["original_doc"]))
    for q in d["questions"]:
        questions.append(Document(metadata={"doc_id": doc_id}, page_content=q))

In [33]:
print(len(docs))
print(len(questions))
print(len(doc_ids))

print(docs[-1])
print(questions[-1])
print(doc_ids[-1])

50
139
50
page_content='chlordiazepoxide is also used to treat irritable bowel syndrome. talk to your doctor about the possible risks of using Chlordiazepoxide for your condition.Chlordiazepoxide is sometimes prescribed for other uses; ask your doctor or pharmacist for more information.' metadata={'doc_id': 'cac34868-f689-43f7-8bd5-09a8346c6cb8'}
page_content='What is Chlordiazepoxide used for?' metadata={'doc_id': 'cac34868-f689-43f7-8bd5-09a8346c6cb8'}
cac34868-f689-43f7-8bd5-09a8346c6cb8


In [34]:
retriever.vectorstore.add_documents(questions)
retriever.docstore.mset(list(zip(doc_ids,docs)))

In [50]:
retriever.vectorstore.similarity_search("what special dietary instructions should i follow about Phenylephrine", k = 5)

[Document(id='51ad4230-f8ba-45fb-82ab-8346fbeca2cf', metadata={'doc_id': '88423ccc-adf6-4ecb-9d4a-48ead1e5f2b7'}, page_content='what special dietary instructions should i follow about Phenylephrine'),
 Document(id='9de1dc96-b6bd-4e31-b3fc-3d85b5e71924', metadata={'doc_id': '88423ccc-adf6-4ecb-9d4a-48ead1e5f2b7'}, page_content='Are there any dietary instructions while using Phenylephrine?'),
 Document(id='8777aca2-7d1e-40dd-9d69-4f0650b17742', metadata={'doc_id': '0ff6d87b-c64f-4ac8-853b-d143cb09a386'}, page_content='Are there any dietary instructions while using Phenylephrine?'),
 Document(id='115c3e53-b666-4395-9da1-ae171e290929', metadata={'doc_id': '22a4ba55-1113-4c1d-9db3-f7eaabb7999a'}, page_content='what special precautions should i follow about Phenylephrine'),
 Document(id='6c9a241f-b657-45e7-89c5-fc1c318ada4d', metadata={'doc_id': '0ff6d87b-c64f-4ac8-853b-d143cb09a386'}, page_content='what other information should i know about Phenylephrine')]

In [44]:
retriever.get_relevant_documents("what special dietary instructions should i follow about Phenylephrine", k = 5)

[Document(metadata={'doc_id': '88423ccc-adf6-4ecb-9d4a-48ead1e5f2b7'}, page_content='unless your doctor tells you otherwise, continue your normal diet.about Phenylephrine'),
 Document(metadata={'doc_id': '0ff6d87b-c64f-4ac8-853b-d143cb09a386'}, page_content='ask your pharmacist any questions you have about phenylephrine.keep a written list of all of the prescription and nonprescription (over-the-counter) medicines, vitamins, minerals, and dietary supplements you are taking. bring this list with you each time you visit a doctor or if you are admitted to the hospital. you should carry the list with you in case of emergencies.about Phenylephrine'),
 Document(metadata={'doc_id': '22a4ba55-1113-4c1d-9db3-f7eaabb7999a'}, page_content='before taking phenylephrine,tell your doctor and pharmacist if you are allergic to phenylephrine, any other medications, or any of the ingredients in phenylephrine preparations.do not take phenylephrine if you are taking a monoamine oxidase (mao) inhibitor, suc

In [51]:
retriever.invoke("what special dietary instructions should i follow about Phenylephrine")

[Document(metadata={'doc_id': '88423ccc-adf6-4ecb-9d4a-48ead1e5f2b7'}, page_content='unless your doctor tells you otherwise, continue your normal diet.about Phenylephrine'),
 Document(metadata={'doc_id': '0ff6d87b-c64f-4ac8-853b-d143cb09a386'}, page_content='ask your pharmacist any questions you have about phenylephrine.keep a written list of all of the prescription and nonprescription (over-the-counter) medicines, vitamins, minerals, and dietary supplements you are taking. bring this list with you each time you visit a doctor or if you are admitted to the hospital. you should carry the list with you in case of emergencies.about Phenylephrine'),
 Document(metadata={'doc_id': '22a4ba55-1113-4c1d-9db3-f7eaabb7999a'}, page_content='before taking phenylephrine,tell your doctor and pharmacist if you are allergic to phenylephrine, any other medications, or any of the ingredients in phenylephrine preparations.do not take phenylephrine if you are taking a monoamine oxidase (mao) inhibitor, suc

3